In [1]:
# Boiler Plate
import findspark
findspark.init('/home/michael/spark-2.1.0-bin-hadoop2.7/')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

Create a model that can predict whether a customer will churn (0 or 1) based off the features

In [4]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
final_data = output.select('features','churn')

In [21]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [22]:
from pyspark.ml.classification import LogisticRegression

In [23]:
lr_churn = LogisticRegression(labelCol='churn')

In [24]:
fitted_churn_model = lr_churn.fit(train_churn)

In [25]:
training_sum = fitted_churn_model.summary

In [26]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                645|                645|
|   mean|0.16589147286821707|0.12558139534883722|
| stddev| 0.3722718827190687| 0.3316341689476192|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [29]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.76806830379027...|[0.99885140699196...|       0.0|
|[28.0,11245.38,0....|    0|[4.06472090611488...|[0.98312197813914...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.84497424819806...|[0.94505831311165...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.06391758862663...|[0.99371895207461...|       0.0|
|[29.0,11274.46,1....|    0|[4.61257798607681...|[0.99017136525161...|       0.0|
|[29.0,13240.01,1....|    0|[6.89094350693918...|[0.99898407925938...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.17850259768997...|[0.96001722910898...|       0.0|
|[30.0,10183.98,1....|    0|[2.98057434550507...|[0.95168878495376...|       0.0|
|[30.0,10960.52,1....|    0|[2.49250947090768...|[0.92361503489277...|       0.0|
|[31.0,5387.75,0

In [34]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [35]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [36]:
# Value of 0.5 would just be as good as guessing
auc

0.7880100921456779

Now let's predict on new data...

In [37]:
final_lr_model = lr_churn.fit(final_data)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/michael/spark-2.1.0-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'BinaryClassificationEvaluator' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/michael/spark-2.1.0-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'BinaryClassificationEvaluator' object has no attribute '_java_obj'


In [38]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [39]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [40]:
test_new_customers = assembler.transform(new_customers)

In [41]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [42]:
final_results = final_lr_model.transform(test_new_customers)

In [43]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|  

In [44]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [46]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          

These are the customers we expect to churn.